In [1]:
import pandas as pd
import numpy as np
import time
import re
from tqdm import tqdm_notebook
from torrequest import TorRequest
from bs4 import BeautifulSoup
from retrying import retry
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ECimport 
import pandas as pd
import souper_v2
from tqdm import tqdm_notebook

In [2]:
sampled_players = pd.read_csv('C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Basic_info_player/basic_info_with_link.csv')
sampled_players

,Unnamed: 0,Unnamed: 0_x,link,player,team,Unnamed: 0_y,name,position,height,age
0,0,0,/david-de-gea/profil/spieler/59377,David de Gea,Manchester United,0,David de Gea,Goalkeeper,189.0,32.0
1,1,1,/jack-butland/profil/spieler/128899,Jack Butland,Manchester United,1,Jack Butland,Goalkeeper,196.0,30.0
2,2,2,/tom-heaton/profil/spieler/34130,Tom Heaton,Manchester United,2,Tom Heaton,Goalkeeper,188.0,37.0
3,3,3,/lisandro-martinez/profil/spieler/480762,Lisandro Martínez,Manchester United,3,Lisandro Martínez,Defender Centre-Back,175.0,25.0
4,4,4,/raphael-varane/profil/spieler/164770,Raphaël Varane,Manchester United,4,Raphaël Varane,Defender Centre-Back,191.0,30.0
...,...,...,...,...,...,...,...,...,...,...
10749,10749,22,/aidan-simmons/profil/spieler/867763,Aidan Simmons,Western Sydney Wanderers,10749,Aidan Simmons,Attack-RightWinger,NaN,20.0
10750,10750,23,/kusini-yengi/profil/spieler/708099,Kusini Yengi,Western Sydney Wanderers,10750,Kusini Yengi,Attack Centre-Forward,190.0,24.0
10751,10751,24,/nathanael-blair/profil/spieler/1023268,Nathanael Blair,Western Sydney Wanderers,10751,Nathanael Blair,Attack Centre-Forward,NaN,19.0
10752,10752,25,/zachary-sapsford/profil/spieler/703657,Zachary Sapsford,Western Sydney Wanderers,10752,Zachary Sapsford,Attack Centre-Forward,NaN,20.0


In [3]:
# Backup of original simplier function to get_soup
def get_soup(URL):
    
    chrome_options = Options()
    chrome_options.add_argument('--window-size=1,1')
    
    #options = webdriver.ChromeOptions()
    #options.add_argument('--disable-popup-blocking')
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)
    wait = WebDriverWait(driver, 10)
    driver.get(URL)
    if len(driver.find_elements_by_css_selector("[name=ROBOTS]")) > 0:
        driver.get(URL)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

In [17]:
# Thu thập dữ liệu chuyên môn của từng cầu thủ
def obtain_players_stats(players_df):
    
    columns = ['player', 'appearance', 'goals', 'assists', 'yellow cards',
               'second yellow cards', 'red cards', 'goals conceded', 'clean sheets', 'minutes played']
    
    stats_player_df = pd.DataFrame(columns = columns)

    for index in range(players_df.shape[0]):
        stats_each_df = pd.DataFrame(columns = columns)
        
        results = 0
        each_link = players_df.iloc[index]['link']
        
        new_link = each_link.replace("profil","leistungsdatendetails")
        new_link = 'https://www.transfermarkt.co.uk' + new_link
        print(new_link)
        
        soup = get_soup(new_link)
        table = soup.find('table', class_ = 'items')
        
        if table is None or new_link == 'https://www.transfermarkt.co.uk/mohammed-jaseem/leistungsdatendetails/spieler/686068':
            stats = {'player':[], 'appearance': [], 'goals': [], 'assists': [], 'yellow cards': [],
            'second yellow cards': [], 'red cards': [], 'goals conceded': [], 'clean sheets': [], 'minutes played': []}
            
            stats['player'].append(each_link)
            stats['appearance'].append(0)
            stats['goals'].append(0)
            stats['assists'].append(0)
            stats['goals conceded'].append(0)
            stats['clean sheets'].append(0)
            stats['yellow cards'].append(0)
            stats['second yellow cards'].append(0)
            stats['red cards'].append(0)
            stats['minutes played'].append(0)
            
            stats1 = pd.DataFrame(stats)
            
            stats_player_df = pd.concat([stats_player_df, stats1])
            
            continue
        
        body = table.find('tbody')
        
        for row in body.find_all('tr'):
            stats = {'player':[], 'season': [], 'appearance': [], 'goals': [], 'assists': [], 'yellow cards': [],
            'second yellow cards': [], 'red cards': [], 'goals conceded': [], 'clean sheets': [], 'minutes played': []}
            
            stats['player'].append(each_link)
            
            td = row.find_all('td')
            
            stats['season'].append(td[0].text)
            stats['appearance'].append(int(td[4].find('a').text.replace('-','0')))
            stats['goals'].append(int(td[5].text.replace('-','0')))
            card_index = 0
        
            if players_df.iloc[index]['position'] == 'Goalkeeper':
                stats['assists'].append(0)
                stats['goals conceded'].append(int(td[7].text.replace('-','0')))
                stats['clean sheets'].append(int(td[8].text.replace('-','0')))
                # Minutes played
                minutes = td[9].text.strip()
                minutes = minutes.replace("'",'')
                minutes = minutes.replace(".",'')
                minutes = minutes.replace('-','0')
                stats['minutes played'].append(int(minutes))
                card_index = 6
                
            else:
                stats['goals conceded'].append(0)
                stats['clean sheets'].append(0)
                stats['assists'].append(int(td[6].text.replace('-','0')))
                # Minutes played
                minutes = td[8].text.strip()
                minutes = minutes.replace("'",'')
                minutes = minutes.replace(".",'')
                minutes = minutes.replace('-','0')
                stats['minutes played'].append(int(minutes))
                card_index = 7
            
            card = td[card_index].text.strip()
            pattern = r"([^/]+)/([^/]+)/([^/]+)"
            matches = re.findall(pattern, card)
            for match in matches:
                a,b,c = match
                stats['yellow cards'].append(int(a.replace('-','0')))
                stats['second yellow cards'].append(int(b.replace('-','0')))
                stats['red cards'].append(int(c.replace('-','0')))
            
            stats_df = pd.DataFrame.from_dict(stats, orient='index')
            stats_df = stats_df.transpose()
            
            stats_each_df = pd.concat([stats_each_df, stats_df])
            
        index_to_drop = []
        
        for i in range(stats_each_df.shape[0]):
            if stats_each_df.iloc[i]['season'] not in ('22/23', '21/22'):
                index_to_drop.append(i)
        #stats_each_df =  stats_each_df.drop(index_to_drop)
        stats_each_df_dropped = stats_each_df.drop('season', axis = 1)
        
        stats = pd.DataFrame(columns = columns)
        
        stats = pd.concat([stats, stats_each_df_dropped.iloc[0].to_frame().T], ignore_index=True)
                
        for i in range(1, stats_each_df.shape[0]):
            if stats_each_df.iloc[i]['season'] in ('22/23', '21/22'):
                stats.iloc[0]['appearance'] += stats_each_df.iloc[i]['appearance']
                stats.iloc[0]['goals'] += stats_each_df.iloc[i]['goals']
                stats.iloc[0]['assists'] += stats_each_df.iloc[i]['assists']
                stats.iloc[0]['yellow cards'] += stats_each_df.iloc[i]['yellow cards']
                stats.iloc[0]['second yellow cards'] += stats_each_df.iloc[i]['second yellow cards']
                stats.iloc[0]['red cards'] += stats_each_df.iloc[i]['red cards']
                stats.iloc[0]['goals conceded'] += stats_each_df.iloc[i]['goals conceded']
                stats.iloc[0]['clean sheets'] += stats_each_df.iloc[i]['clean sheets']
                stats.iloc[0]['minutes played'] += stats_each_df.iloc[i]['minutes played']
       
        # Transform the data to per90
        per90 = stats.iloc[0]['minutes played']/90
        if per90 != 0:
            stats.iloc[0]['goals'] /= per90
            stats.iloc[0]['assists'] /= per90
            stats.iloc[0]['yellow cards'] /= per90
            stats.iloc[0]['second yellow cards'] /= per90
            stats.iloc[0]['red cards'] /= per90
            stats.iloc[0]['goals conceded'] /= per90
            stats.iloc[0]['clean sheets'] /= per90
        
        
        stats_player_df = pd.concat([stats_player_df, stats])
    
    return stats_player_df
            

In [5]:
# Sample_data là một list, mỗi phần tử là 1 dataframe tách ra từ sampled_players
sample_data = []
sample_data.append(sampled_players.iloc[:10])
i = 10
while i < sampled_players.shape[0]:
    if i >= sampled_players.shape[0] - 200:
        sample_data.append(sampled_players.iloc[i:])
    else:
        sample_data.append(sampled_players.iloc[i:i+200])
    i += 200 # mỗi sample chứa 200 dòng
print(len(sample_data))

55


In [19]:
i = 46
sample = obtain_players_stats(sample_data[i])
sample

https://www.transfermarkt.co.uk/aschraf-el-mahdioui/leistungsdatendetails/spieler/295764


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.110)


In [20]:
file_name = "C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Player_stats/sample_data_" + str(i) + ".csv"
sample.to_csv(file_name) # Lưu thành file sample_data_i.csv

In [21]:
sample


,player,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,minutes played
0,/aschraf-el-mahdioui/profil/spieler/295764,63,0.047855,0.015952,0.255229,0.0,0.0,0.0,0.0,5642
0,/alvaro-medran/profil/spieler/253839,47,0.152247,0.086999,0.326245,0.0,0.0,0.0,0.0,4138
0,/flavio/profil/spieler/373191,47,0.022466,0.089865,0.089865,0.0,0.0,0.0,0.0,4006
0,/kaku/profil/spieler/355817,43,0.343075,0.543202,0.171537,0.0,0.0,0.0,0.0,3148
0,/saad-al-nasser/profil/spieler/888634,23,0.245009,0.326679,0.245009,0.0,0.0,0.0,0.0,1102
...,...,...,...,...,...,...,...,...,...,...
0,/lucas-souza/profil/spieler/216259,64,0.11115,0.11115,0.190543,0.031757,0.0,0.0,0.0,5668
0,/hassan-al-majhad/profil/spieler/1051403,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,397
0,/ayman-al-hujaili/profil/spieler/209749,17,0.0,0.0,0.148026,0.0,0.0,0.0,0.0,608
0,/hamad-al-abdan/profil/spieler/883869,34,0.06237,0.0,0.06237,0.0,0.0,0.0,0.0,1443
